# 뉴스 데이터 유사도 측정 최종본

Newspaper_Similarity.ipynb (소장님코드) 를 보고 main.ipynb 를 고친 최종코드

2021.7.13 완료

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## 데이터 불러오기

In [ ]:
df = pd.read_csv('./pretreatment_data/yeonhap_pre.csv', names=['CNo', 'Subject', 'Contents'])
df.info()

In [ ]:
et_df = pd.read_csv('./pretreatment_data/etoday_pre.csv', names=['CNo', 'Subject', 'Contents'])
et_df.info()

## TfidfVectorizer() 함수 객체 선언

fit_transform 으로 연합 뉴스 데이터에 대한 설정값을 만들어 둔다.

이후 transform 만 사용하면 연합 뉴스 데이터 셋으로 학습된 설정값을 그대로 이용 가능하다.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

tfidf_matrix = tfidf.fit_transform(df['Contents'])
print(tfidf_matrix.shape)

In [ ]:
target_matrix = tfidf.transform(et_df['Contents'])
print(target_matrix.shape)

In [ ]:
et_df._get_value(0, 'Contents')

## 유사도 계산 함수 생성

- 매개변수로 num이 들어가는데, 이는 line 번호를 의미한다. 

- cosine_similarity(문서 1, 문서2)

        - 문서 1 자리에는 새로 만든 문서(이투데이)를 넣고, 문서 2에는 비교할 문서(연합)들을 넣는다.

        - 코사인 유사도 계산를 통해 문서 2들 중에서 문서 1과 유사도가 높게 나온 것들을 뽑는다.

In [ ]:
def clac_sim(num):
    # target_matrix에는 문서 하나(이투데이)의 TF-IDF 값이 저장된다.
    target_matrix = tfidf.transform([et_df._get_value(num, 'Contents')]).astype(np.float16)
    # cosin_matrix에는 유사도 결과 값들이 저장된다.
    cosine_matrix = cosine_similarity(target_matrix, tfidf_matrix)

# 보기 쉽게 맵핑하는 부분
    # news title과 id를 맵핑할 dictionary를 생성
    news2id = {}
    for i, c in enumerate(df['CNo']):
        news2id[i] = c

    # id와 news title를 매핑할 dictionary를 생성
    id2news = {}
    for i, c in news2id.items():
        id2news[c] = i
# 맵핑 끝
    sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[0])]
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
    # sim_scores 에는 문서 번호(CNo) 와 유사도 값이 리스트 형태로 저장됨
    # 5개 저장
    sim_scores = [(news2id[i], score) for i, score in sim_scores[0:5]]

    return sim_scores

## 동작 부분

In [ ]:
# 문서(이투데이) 총 라인 수가 9747
# 숫자로 표기하지 말고 len(et_df) 이렇게 표기하는게 훨씬 보기도 좋고 직관적임.
for num in range(len(et_df)):
    sim_scores = clac_sim(num)
    sim = []
    # 유사도가 90% 이상인 뉴스들만 저장한다.
    if sim_scores[0][1] > 0.9:
        sim.append(sim_scores[0][0])
        result = pd.DataFrame({
            'CNo' : et_df.iloc[num]['CNo'],
            'Subject' : et_df.iloc[num]['Subject'],
            'similar' : sim
        })
        # 바로 저장
        result.to_csv("./similar_result_90_change.csv", encoding='utf-8',mode = 'a', index=False, header=False)

## 개선할 점

1. 바로 위에 코드를 보면 5개중 하나만(제일 유사도가 높은 것) 만 저장이 된다. 코드 수정을 통해 5개를 반복문으로 훝어 90% 이상인 문서들을 저장할 필요가 있다.
2. 소장님 코드를 보면 속도 향상을 위해 cosin_similarity를 안쓰고 직접 계산 했다. 비교해보고 진짜 시간이 적게 걸리는지 알아볼 필요가 있다.

# 끝